In [ ]:
#best result:  0.8730396297745561
#best parameters:  [0.13902060171594238, 26, 10543, 0.8018499007715856, 0.2104357776007047]

#best result:  0.8807704974617095
#best parameters:  [0.040379975519599896, 35, 14376, 0.7728573204303828, 0.17804513595302274]

#best result:  0.879092110010965
#best parameters:  [0.047741362010590316, 23, 10080, 0.8598644148897474, 0.4543541558368278]

import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn import metrics

In [ ]:
#
# Prepare the data
#

train = pd.read_csv("duom/train_data_2.csv")
test_data = pd.read_csv("duom/test_data_2.csv")

le1 = preprocessing.LabelEncoder()
le2 = preprocessing.LabelEncoder()
cols = ['term', 'credit_score', 'loan_purpose', 'home_ownership']
for col in cols:
    train[col] = le1.fit_transform(train[col])
    test_data[col] = le2.fit_transform(test_data[col])
    print(le1.classes_)
    print(le2.classes_)

y = train['y']
x = train.drop(['y', 'id'], axis=1)

In [ ]:
#
# Create training and validation sets
#
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

#
# Create the LightGBM data containers
#
lgb_train = lgb.Dataset(x_train, label=y_train)
lgb_eval = lgb.Dataset(x_test, label=y_test, reference=lgb_train)

In [ ]:
"""params = {
    'lambda_l1': 0.0004, 
    'lambda_l2': 0.004, 
    'num_leaves': 30000, 
    'feature_fraction': 0.8, 
    'bagging_fraction': 0.8, 
    'bagging_freq': 2, 
    'min_child_samples': 13,
}

print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                callbacks=[lgb.early_stopping(stopping_rounds=5)])

preds = gbm.predict(x_test)
pred_labels = np.rint(preds)
accuracy = metrics.accuracy_score(y_test, pred_labels)
print(f'The ACU of prediction is: {accuracy}')"""

#0.8431695524810852
#0.8424313841647153

In [ ]:
"""ids = test_data['id'].values
test_data = test_data.drop(['id'], axis=1)
x = test_data.values
y = gbm.predict(x)
output = pd.DataFrame({'id': ids, 'target': y})"""

In [8]:
SEARCH_PARAMS = {'learning_rate': 0.040379975519599896, 
                'max_depth': 35, 
                'num_leaves': 14376, 
                'feature_fraction': 0.965292624452857, 
                'subsample': 0.17804513595302274}

def train_evaluate(search_params):
    params = {'objective': 'binary',
              'metric': 'auc',
              **search_params}

    model = lgb.train(params, lgb_train,
                      num_boost_round=500,
                      #early_stopping_rounds=10,
                      callbacks=[lgb.early_stopping(stopping_rounds=50)],
                      valid_sets=lgb_eval,
                      valid_names='valid'
                     )
    
    score = model.best_score['valid']['auc']
    return score, model

if __name__ == '__main__':
    score, model = train_evaluate(SEARCH_PARAMS)
    print('validation AUC:', score)

[LightGBM] [Info] Number of positive: 1431971, number of negative: 4568269
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.227628 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1541
[LightGBM] [Info] Number of data points in the train set: 6000240, number of used features: 15
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.238652 -> initscore=-1.160083
[LightGBM] [Info] Start training from score -1.160083
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[265]	valid's auc: 0.883943
validation AUC: 0.883943337994814


In [9]:
ids = test_data['id'].values
test_data = test_data.drop(['id'], axis=1)

In [10]:
x = test_data.values
y = model.predict(x)
output = pd.DataFrame({'id': ids, 'target': y})
output.head(15)

,id,target
0,1,0.256321
1,2,0.441151
2,3,0.042678
3,4,0.242448
4,5,0.260231
5,6,0.092428
6,7,0.000010
7,8,0.240470
8,9,0.000007
9,10,0.541795


In [11]:
output.to_csv("submission_2_GBM.csv", index=False)

In [12]:
model.save_model('model_2.txt')

In [ ]:
"""import skopt

SPACE = [
    skopt.space.Real(0.01, 0.4, name='learning_rate', prior='log-uniform'),
    skopt.space.Integer(1, 40, name='max_depth'),
    skopt.space.Integer(8000, 18000, name='num_leaves'),
    skopt.space.Real(0.5, 1.0, name='feature_fraction', prior='uniform'),
    skopt.space.Real(0.1, 0.6, name='subsample', prior='uniform')]
@skopt.utils.use_named_args(SPACE)
def objective(**params):
    return -1.0 * train_evaluate(params)
results = skopt.forest_minimize(objective, SPACE, n_calls=20, n_random_starts=10)
best_auc = -1.0 * results.fun
best_params = results.x

print('best result: ', best_auc)
print('best parameters: ', best_params)"""